In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [15]:
import math
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, classes=10, kernel = 3, channels=3):
        super(Net, self).__init__()
        self.kernel = kernel
        self.channels = channels
        self.conv = nn.Conv2d(3,self.channels,self.kernel,1) 
        self.pad = math.ceil((32 - (32 - self.kernel + 1))/2)
        self.linear = nn.Linear(self.channels*(32 - self.kernel + 1 + self.pad*2)*(32 - self.kernel + self.pad*2 + 1),10)
        self.flat = nn.Flatten()

    def forward(self, x):
        x = torch.nn.functional.pad(x,(self.pad,self.pad,self.pad,self.pad),'circular')
        x = self.conv(x)
        x = self.flat(x)
        x = self.linear(x)
        return x
    
class Net3(nn.Module):
    def __init__(self, classes=10, kernel = 3, channels=3):
        super(Net3, self).__init__()
        self.kernel = kernel
        self.channels = channels
        
        self.conv1 = nn.Conv2d(3,self.channels,self.kernel,1) 
        self.pad1 = math.ceil((32 - (32 - self.kernel + 1))/2)
        self.output1 = int((32 - self.kernel + 1 + self.pad1*2))
        
        self.conv2 = nn.Conv2d(self.channels, self.channels, self.kernel,1) 
        self.pad2 = math.ceil((self.output1 - (self.output1 - self.kernel + 1))/2)
        self.output2 = int((self.output1 - self.kernel + 1 + self.pad2*2))
        
        self.conv3 = nn.Conv2d(self.channels, self.channels, self.kernel,1) 
        self.pad3 = math.ceil((self.output2 - (self.output2 - self.kernel + 1))/2)
        self.output3 = int((self.output2 - self.kernel + 1 + self.pad3*2))
        
        self.linear = nn.Linear(self.channels*(self.output3 - self.kernel + 1 + self.pad3*2)*(self.output3 - self.kernel + self.pad3*2 + 1),10)
        self.flat = nn.Flatten()

    def forward(self, x):
        x = torch.nn.functional.pad(x,(self.pad1,self.pad1,self.pad1,self.pad1),'circular')
        x = self.conv1(x)
        x = torch.nn.functional.pad(x,(self.pad2,self.pad2,self.pad2,self.pad2),'circular')
        x = self.conv2(x)
        x = torch.nn.functional.pad(x,(self.pad3,self.pad3,self.pad3,self.pad3),'circular')
        x = self.conv3(x)
        x = self.flat(x)
        x = self.linear(x)
        return x
    


posible_models = ['fclinearl3', 'fclinearl1','convlinearl1k3c3', 'convlinearl3k3c3','convlinearl1k11c3','convlinearl3k11c3','convlinearl1k32c3',
                 'convlinearl3k32c3','convlinearl1k3c8','convlinearl1k3c32','convlinearl3k3c8','convlinearl3k3c32']

def return_model(name='fclinearl1'):
    if name == 'fclinearl3':
        model = nn.Sequential(nn.Flatten(), nn.Linear(3*32*32,3*32*32),nn.Linear(3*32*32,3*32*32), nn.Linear(3*32*32,3*32*32), nn.Linear(3*32*32,10))
    elif name == 'fclinearl1':
        model = nn.Sequential(nn.Flatten(),nn.Linear(3*32*32,3*32*32), nn.Linear(3*32*32,10))
    elif name == 'convlinearl1k3c3':
        model = Net(kernel=3,channels=3)
    elif name == 'convlinearl3k3c3':
        model = Net3(kernel=3,channels=3)
    elif name == 'convlinearl1k11c3':
        model = Net(kernel=11,channels=3)
    elif name == 'convlinearl3k11c3':
        model = Net3(kernel=11, channels=3)
    elif name == 'convlinearl1k32c3':
        model = Net(kernel=32, channels=3)
    elif name == 'convlinearl3k32c3':
        model = Net3(kernel=32, channels=3)
    elif name == 'convlinearl1k3c8':
        model = Net(kernel=3,channels=8)
    elif name == 'convlinearl1k3c32':
        model = Net(kernel=3, channels=32)
    elif name == 'convlinearl3k3c8':
        model = Net3(kernel=3, channels=8)
    elif name == 'convlinearl3k3c32':
        model = Net3(kernel=3, channels=32)
        
    return model

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os


#parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
#parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
#arser.add_argument('--resume', '-r', action='store_true',
#                    help='resume from checkpoint')
#args = parser.parse_args()

args = {}
args['resume'] = False
args['lr'] = 0.01

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

posible_models = ['fclinearl3', 'fclinearl1','convlinearl1k3c3', 'convlinearl3k3c3','convlinearl1k11c3','convlinearl3k11c3','convlinearl1k32c3',
                 'convlinearl3k32c3','convlinearl1k3c8','convlinearl1k3c32','convlinearl3k3c8','convlinearl3k3c32']
# Model
print('==> Building model..')
name = 'convlinearl1k3c32'
net = return_model(name)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if args['resume']:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args['lr'],
                      momentum=0.9, weight_decay=5e-4)


lambda1 = lambda epoch: 0.3**((epoch)//4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR()
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

def train(epoch, name):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        #if epoch == 0 and batch_idx == 0:
        if batch_idx%30 == 0:
            torch.save(net.state_dict(), './cifar_resnet/ckpt_{}_{}_{}.pth'.format(name,epoch, batch_idx+1))
        #torch.save(net.state_dict(), './cifar_resnet/ckpt_fc_{}_{}.pth'.format(epoch, batch_idx))
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx%100 == 0:
            print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'%(train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch,name):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('cifar_resnet'):
            os.mkdir('cifar_resnet')
        torch.save(state, './cifar_resnet/ckpt_{}.pth'.format(name))
        best_acc = acc


for epoch in range(start_epoch, start_epoch+20):
    train(epoch,name)
    test(epoch,name)
    scheduler.step()

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..

Epoch: 0
0 391 Loss: 2.307 | Acc: 13.281% (17/128)
100 391 Loss: 1.936 | Acc: 32.758% (4235/12928)
200 391 Loss: 1.874 | Acc: 34.958% (8994/25728)
300 391 Loss: 1.841 | Acc: 36.215% (13953/38528)
99 100 Loss: 1.737 | Acc: 39.820% (3982/10000)
Saving..

Epoch: 1
0 391 Loss: 1.736 | Acc: 42.969% (55/128)
100 391 Loss: 1.723 | Acc: 41.136% (5318/12928)
200 391 Loss: 1.740 | Acc: 40.749% (10484/25728)
300 391 Loss: 1.737 | Acc: 40.737% (15695/38528)
99 100 Loss: 1.724 | Acc: 41.230% (4123/10000)
Saving..

Epoch: 2
0 391 Loss: 1.707 | Acc: 37.500% (48/128)
100 391 Loss: 1.702 | Acc: 42.041% (5435/12928)
200 391 Loss: 1.699 | Acc: 41.931% (10788/25728)
300 391 Loss: 1.706 | Acc: 41.697% (16065/38528)
99 100 Loss: 1.737 | Acc: 39.740% (3974/10000)

Epoch: 3
0 391 Loss: 1.555 | Acc: 45.312% (58/128)
100 391 Loss: 1.667 | Acc: 43.038% (5564/12928)
200 391 Loss: 1.686 | Acc: 42.